In [ ]:
import os
import pymysql
conn = pymysql.connect(
    host=".",
    port=.,
    user="lgup2",
    passwd="lgup2P@ssw0rd",
    database="lgup2",
    local_infile=True,
    cursorclass=pymysql.cursors.DictCursor
)

try: 
    with conn.cursor() as cur:
        # 2) 세션 문자셋 재확인(전송 구간 깨짐 방지)
        cur.execute("SET NAMES utf8mb4;")
        sql = """
        SELECT      *
        FROM        k_contents_trans
        
        """
        
        cur.execute(sql)
        # 4) 결과 가져오기
        rows = cur.fetchall() # list[dict] / [{id:1}, {location_name:남산타워}]
finally:
    conn.close()

In [3]:
import pandas as pd
df = pd.DataFrame(rows)

In [5]:
df.columns

Index(['content_id', 'drama_name', 'location_name', 'address', 'trip_tip',
       'keyword', 'category', 'location_name_en', 'drama_name_en',
       'address_en', 'category_en', 'keyword_en', 'image_url',
       'image_url_list', 'thumbnail', 'second_image', 'third_image',
       'latitude', 'longitude', 'drama_desc', 'trip_tip_en'],
      dtype='object')

In [11]:
df.drop(columns = ['location_name', 'address', 'trip_tip',
       'keyword', 'category','image_url',
       'image_url_list', 'drama_desc'], inplace = True)

In [12]:
df.columns

Index(['content_id', 'drama_name', 'location_name_en', 'drama_name_en',
       'address_en', 'category_en', 'keyword_en', 'thumbnail', 'second_image',
       'third_image', 'latitude', 'longitude', 'trip_tip_en'],
      dtype='object')

In [8]:
import os
import pandas as pd
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
print("✅ 라이브러리 임포트 완료!")

c:\Users\Admin\miniconda3\envs\rag-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 라이브러리 임포트 완료!


In [ ]:
os.environ["OPENAI_API_KEY"] = "."  # ← 수정!

# 파일 및 설정
QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "seoul-kcontents"
                                                                                                                                                                                                              
print("✅ 설정 완료!")

✅ 설정 완료!


In [13]:
import pandas as pd
from langchain_core.documents import Document

# CSV_FILE 변수와 df (DataFrame) 변수는 외부에서 정의되었다고 가정합니다.

print("📂 CSV 파일 로딩 및 전처리 중...")


print(f"✅ 총 {len(df)}개 항목 로드")
print(f"\n컬럼: {df.columns.tolist()}")

# 의미 있는 Document 생성
docs = []
for idx, row in df.iterrows():
    # 1. 임베딩할 텍스트 (page_content) 생성 - 검색 성능 향상을 위해 중요한 정보만 조합
    # 검색용 텍스트에는 한글 이름과 영문 이름을 모두 포함하여 검색 범위를 넓힙니다.
    text = f"""
드라마/쇼(한글): {row['drama_name']}
촬영지(영문): {row['location_name_en']}
드라마/쇼(영문): {row['drama_name_en']}
주소(영문): {row['address_en']}
카테고리: {row['category_en']}
키워드: {row['keyword_en']}
    """.strip()
    
    # 2. Document 생성
    doc = Document(
        page_content=text,
        metadata={
            # ID 및 검색/필터링에 자주 쓰일 핵심 컬럼
            'content_id': str(row['content_id']),
            'drama_name_ko': str(row['drama_name']), # 한글 드라마 이름
            'location_name_en': str(row['location_name_en']), # 촬영지 영문 이름
            'address_en': str(row['address_en']),
            'latitude': float(row['latitude']),
            'longitude': float(row['longitude']),
            
            # 모든 나머지 컬럼 (Payload에 저장됨)
            'drama_name_en': str(row['drama_name_en']),
            'category_en': str(row['category_en']),
            'keyword_en': str(row['keyword_en']),
            'trip_tip_en': str(row['trip_tip_en']),
            'thumbnail': str(row['thumbnail']),
            'second_image': str(row['second_image']),
            'third_image': str(row['third_image']),
        }
    )
    docs.append(doc)

print(f"✅ {len(docs)}개 Document 생성 완료!")
print(f"\n첫 번째 항목 샘플:")
print(docs[0].page_content)

📂 CSV 파일 로딩 및 전처리 중...
✅ 총 1184개 항목 로드

컬럼: ['content_id', 'drama_name', 'location_name_en', 'drama_name_en', 'address_en', 'category_en', 'keyword_en', 'thumbnail', 'second_image', 'third_image', 'latitude', 'longitude', 'trip_tip_en']
✅ 1184개 Document 생성 완료!

첫 번째 항목 샘플:
드라마/쇼(한글): 이혼보험
촬영지(영문): Jongno 1st Street Opistel
드라마/쇼(영문): divorce insurance
주소(영문): 67 Cheonggyecheon-ro, Jongno-gu, Seoul
카테고리: Other/Not distinguishable
키워드: #Divorce insurance #Jongro 1st Street Opistel #filming location #modern architecture #Seoultravel


In [14]:
print("🤖 OpenAI 임베딩 모델 준비 중...")

embedding = OpenAIEmbeddings(model="text-embedding-ada-002")

print("✅ 임베딩 모델 준비 완료!")

🤖 OpenAI 임베딩 모델 준비 중...
✅ 임베딩 모델 준비 완료!


In [15]:
# 셀 5: Qdrant 연결 및 컬렉션 생성
print("🔌 Qdrant 연결 중...")

qdrant_client = QdrantClient(url=QDRANT_URL)

# 기존 컬렉션 삭제
try:
    qdrant_client.delete_collection(collection_name=COLLECTION_NAME)
    print("🗑️  기존 컬렉션 삭제됨")
except:
    print("ℹ️  기존 컬렉션 없음")

# 새 컬렉션 생성
qdrant_client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
)

print(f"✅ 컬렉션 '{COLLECTION_NAME}' 생성 완료!")

🔌 Qdrant 연결 중...
🗑️  기존 컬렉션 삭제됨
✅ 컬렉션 'seoul-kcontents' 생성 완료!


In [16]:
# 셀 6: 데이터 임베딩 및 저장
print(f"\n💾 {len(docs)}개 컨텐츠 임베딩 생성 및 저장 중...")
print("⏳ OpenAI API 호출 중... (약 10-15분 소요)")

vector_store = Qdrant.from_documents(
    documents=docs,
    embedding=embedding,
    url=QDRANT_URL,
    collection_name=COLLECTION_NAME,
)

print(f"\n✅ 총 {len(docs)}개 레스토랑 저장 완료!")
print(f"📊 Qdrant UI: {QDRANT_URL}/dashboard")


💾 1184개 컨텐츠 임베딩 생성 및 저장 중...
⏳ OpenAI API 호출 중... (약 10-15분 소요)

✅ 총 1184개 레스토랑 저장 완료!
📊 Qdrant UI: http://localhost:6333/dashboard


In [17]:
# 셀 7: 저장 확인
collection_info = qdrant_client.get_collection(COLLECTION_NAME)

print(f"📊 컬렉션 정보:")
print(f"  - 컬렉션명: {COLLECTION_NAME}")
print(f"  - 저장된 레코드: {collection_info.points_count}개")
print(f"  - 벡터 차원: {collection_info.config.params.vectors.size}")

📊 컬렉션 정보:
  - 컬렉션명: seoul-kcontents
  - 저장된 레코드: 1184개
  - 벡터 차원: 1536


In [22]:
# 셀 8: 검색 함수 정의 (수정됨)
def search_content(query, k=5):
    """엔터테인먼트 검색"""
    print(f"🔍 질의: '{query}'")
    print("=" * 60)
    
    # vector_store는 Qdrant와 연결된 LangChain 벡터 스토어 객체라고 가정합니다.
    results = vector_store.similarity_search(query, k=k)
    
    for i, doc in enumerate(results, 1):
        print(f"\n[{i}위]")
        print("-" * 60)
        
        # 메타데이터 출력 (새로운 컬럼 이름 사용)
        meta = doc.metadata
        print(f"🆔 ID: {meta.get('content_id', 'N/A')}")
        print(f"🎬 제목: {meta.get('drama_name_en', 'N/A')}")
        print(f"📍 주소: {meta.get('address_en', 'N/A')}")
        print(f"⏰ 운영: {meta.get('category_en', 'N/A')}")
        print(f"ℹ️ 중요: {meta.get('keyword_en', 'N/A')}")
        
        print(f"\n내용:")
        # page_content에는 title, address, description, transportation, Hours of Operation, Fee 등이 포함되어 있습니다.
        print(doc.page_content[:200].replace('\n', ' ') + "...")
        print("-" * 60)
    
    return results

print("✅ 검색 함수 준비 완료!")

✅ 검색 함수 준비 완료!


In [23]:
search_content("famous restaurant", k=3)

🔍 질의: 'famous restaurant'

[1위]
------------------------------------------------------------
🆔 ID: 1144
🎬 제목: Will everything come true
📍 주소: 6-minute walk from Exit 3 of Yeouido Station, 10 International Finance-ro, Yeongdeungpo-gu, Seoul
⏰ 운영: a cafe/restaurant
ℹ️ 중요: #Filming location #luxuryrestaurant #steak #urbanview #date #specialexperience

내용:
드라마/쇼(한글): 다 이루어질지니 촬영지(영문): Conrad Seoul 37 Grill & Bar 드라마/쇼(영문): Will everything come true 주소(영문): 6-minute walk from Exit 3 of Yeouido Station, 10 International Finance-ro, Yeongdeungpo-gu, Seoul ...
------------------------------------------------------------

[2위]
------------------------------------------------------------
🆔 ID: 927
🎬 제목: This relationship is irresistible
📍 주소: 7-minute walk from Exit 4 of Apgujeong Rodeo Station, 21, Seonneung-ro 152-gil, Gangnam-gu, Seoul
⏰ 운영: a cafe/restaurant
ℹ️ 중요: #Date #Romantic #Steak #High-end Restaurant #Gangnam

내용:
드라마/쇼(한글): 이 연애는 불가항력 촬영지(영문): Wolfgang Steakhouse 드라마/쇼(영문): This rel

[Document(metadata={'content_id': '1144', 'drama_name_ko': '다 이루어질지니', 'location_name_en': 'Conrad Seoul 37 Grill & Bar', 'address_en': '6-minute walk from Exit 3 of Yeouido Station, 10 International Finance-ro, Yeongdeungpo-gu, Seoul', 'latitude': 37.5249782, 'longitude': 126.9252984, 'drama_name_en': 'Will everything come true', 'category_en': 'a cafe/restaurant', 'keyword_en': '#Filming location #luxuryrestaurant #steak #urbanview #date #specialexperience', 'trip_tip_en': 'Conrad Seoul 37 Grill  Bar is famous for its modern design and luxurious atmosphere as the filming location for the drama Will It All Be Done Here we offer a wide range of menus especially with highquality steaks and you can enjoy great city views Perfect for a special time with friends or lovers comfortable seating and meticulous service are attractive If you are a drama fan it will be a good experience to visit the filming location in person', 'thumbnail': 'https://blog.kakaocdn.net/dna/EvwM3/btsQ4lEQ28J/AAAAAAA